In [1]:
# Read .edf files and save them in a 2D dictionary (subject,round)

import mne
import pyedflib
import numpy as np
import os

path = "../../Dataset/S074"                                  # Read specific subject dataset
file_name = os.path.join(path, 'S074R04.edf')             # Read specific round of the subject

round_data = mne.io.read_raw_edf(file_name)
round_data.load_data()
round_data.rename_channels(lambda s: s.strip("."))

print(round_data.info)

array_rnd_data = round_data.get_data()



Extracting EDF parameters from C:\Users\Alkinoos\Desktop\ML on mCs\Project\Dataset\S074\S074R04.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19839  =      0.000 ...   123.994 secs...
<Info | 7 non-empty values
 bads: []
 ch_names: Fc5, Fc3, Fc1, Fcz, Fc2, Fc4, Fc6, C5, C3, C1, Cz, C2, C4, C6, ...
 chs: 64 EEG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 80.0 Hz
 meas_date: 2009-08-12 16:15:00 UTC
 nchan: 64
 projs: []
 sfreq: 160.0 Hz
>


In [2]:
# Read events from annotations of file - NOT NEEDED FOR THE REST - just for demonstration
events = mne.read_annotations(file_name)

a=getattr(events,'description')
print(a.shape[0])
print("\n")

30




In [4]:
time_sample = np.zeros((625,3))

for i in range(0,625):
    time_sample[i] = [i, i*(0.2), i*0.2+0.4]
    print(time_sample[i])
    

[0.  0.  0.4]
[1.  0.2 0.6]
[2.  0.4 0.8]
[3.  0.6 1. ]
[4.  0.8 1.2]
[5.  1.  1.4]
[6.  1.2 1.6]
[7.  1.4 1.8]
[8.  1.6 2. ]
[9.  1.8 2.2]
[10.   2.   2.4]
[11.   2.2  2.6]
[12.   2.4  2.8]
[13.   2.6  3. ]
[14.   2.8  3.2]
[15.   3.   3.4]
[16.   3.2  3.6]
[17.   3.4  3.8]
[18.   3.6  4. ]
[19.   3.8  4.2]
[20.   4.   4.4]
[21.   4.2  4.6]
[22.   4.4  4.8]
[23.   4.6  5. ]
[24.   4.8  5.2]
[25.   5.   5.4]
[26.   5.2  5.6]
[27.   5.4  5.8]
[28.   5.6  6. ]
[29.   5.8  6.2]
[30.   6.   6.4]
[31.   6.2  6.6]
[32.   6.4  6.8]
[33.   6.6  7. ]
[34.   6.8  7.2]
[35.   7.   7.4]
[36.   7.2  7.6]
[37.   7.4  7.8]
[38.   7.6  8. ]
[39.   7.8  8.2]
[40.   8.   8.4]
[41.   8.2  8.6]
[42.   8.4  8.8]
[43.   8.6  9. ]
[44.   8.8  9.2]
[45.   9.   9.4]
[46.   9.2  9.6]
[47.   9.4  9.8]
[48.   9.6 10. ]
[49.   9.8 10.2]
[50.  10.  10.4]
[51.  10.2 10.6]
[52.  10.4 10.8]
[53.  10.6 11. ]
[54.  10.8 11.2]
[55.  11.  11.4]
[56.  11.2 11.6]
[57.  11.4 11.8]
[58.  11.6 12. ]
[59.  11.8 12.2]
[60.  12. 

In [5]:
a = np.zeros(10)
i=0
for window in (1,5,7,8,9):
    a[i] = window
    i = i+1
    
print(a)

[1. 5. 7. 8. 9. 0. 0. 0. 0. 0.]


In [6]:
# Extract events from annotations and use dictionary to specify required label - IT DEPENDS ON THE ROUND!
# rest: 0 , left fist: 1, right fist: 2, both fists: 3, both feet: 4


# Here Round 4 so T1: right fist, T2: left fist
dictionary = {'T0':0, 'T1':1, 'T2':2}    # Specify label

ev_fr_ann, event_id = mne.events_from_annotations(round_data, event_id=dictionary)
print(ev_fr_ann)           # Events and their labels at the second column
print(ev_fr_ann[0])
print(ev_fr_ann[0][2])     # Code of specific event
print(event_id)

Used Annotations descriptions: ['T0', 'T1', 'T2']
[[    0     0     0]
 [  688     0     2]
 [ 1344     0     0]
 [ 2000     0     1]
 [ 2656     0     0]
 [ 3312     0     1]
 [ 3968     0     0]
 [ 4624     0     2]
 [ 5280     0     0]
 [ 5936     0     2]
 [ 6592     0     0]
 [ 7248     0     1]
 [ 7904     0     0]
 [ 8560     0     2]
 [ 9216     0     0]
 [ 9872     0     1]
 [10528     0     0]
 [11184     0     2]
 [11840     0     0]
 [12496     0     1]
 [13152     0     0]
 [13808     0     2]
 [14464     0     0]
 [15120     0     1]
 [15776     0     0]
 [16432     0     1]
 [17088     0     0]
 [17744     0     2]
 [18400     0     0]
 [19056     0     1]]
[0 0 0]
0
{'T0': 0, 'T1': 1, 'T2': 2}


In [22]:
# Make numpy array to save the labels

import h5py
label_index =0
f = h5py.File("Y_training_1.h5","a")
f.create_dataset('labels', shape=(185400,1), compression="gzip", compression_opts=9)
size = ev_fr_ann.shape
print(size[0])
round_labels = np.zeros((10*size[0],1))

for i in range(0,30):
    round_labels[(10*i):(10*i)+10] = ev_fr_ann[i][2]   # 10 times the label because of 10 windows when extracting images of 0.4s
        # Save labels in the file to be used as training and testing labels
    with h5py.File('./Y_training_1.h5','a') as hf:
        hf["labels"][label_index:(label_index+10)] =  round_labels[(10*i):(10*i)+10]
        label_index = label_index + 10

30


In [23]:
# Show pictures from the read dataset

f = h5py.File("./Y_training_1.h5","r")

data = f['labels'][:]
f.close()

print(data.shape)
print(data[0].shape)

(185400, 1)
(1,)


In [24]:
print(data[0:300])
print("\n")
print(round_labels)

[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.